In [22]:
import numpy as np
import pandas as pd
%pylab inline
import json

Populating the interactive namespace from numpy and matplotlib


C:\Users\divit\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['add']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [23]:
class TrieNode(object):
  
    def __init__(self, char: str):
        self.char = char
        self.children = []
        self.word_finished = False

In [24]:
def add(root, word: str):
    word = word.lower().replace('ё', 'е')  #lower- преобразует все символы в нижний регистр, replace меняет ё на е
    node = root
    for char in word:
        found_in_child = False
        for child in node.children:
            if child.char == char:
                node = child
                found_in_child = True
                break
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            node = new_node
    node.word_finished = True
    
def prefix_descent(root, prefix: str):
    node = root
    for char in prefix:
        char_not_found = True
        for child in node.children:
            if child.char == char:
                char_not_found = False
                node = child
                break
        if char_not_found:
            return False
    return node


def trie_print(node, string, level, list_of_strings):
    if node.word_finished:
        res_string = ''.join(string)
        list_of_strings.append(res_string)
    else:
        for child in node.children:
            string.insert(level, child.char)
            trie_print(child, string.copy(), level+1, list_of_strings)
            string.pop()


def suggester(root, prefix):
    prefix = prefix.lower()
    node = prefix_descent(root, prefix)
    list_of_strings = []
    string = []
    if type(node) == bool:
        list_of_strings.append(prefix)
    else:
        trie_print(node, string, 0, list_of_strings)
        for i in range(len(list_of_strings)):
            list_of_strings[i] = prefix + list_of_strings[i]
    return list_of_strings   
    

In [25]:
def rating(root, word , number: int,data:pd.DataFrame):
    string = suggester(root, word)
    sugg_data = pd.DataFrame(columns =  ["name", "rating"])
    n = len(data)
    i = 0
    while i < n:
        for s in string:
            if data.at[i, 'name'] == s:
                sugg_data = pd.DataFrame(np.array([[data.loc[i]['name'], data.loc[i]['rating']]]), columns=['name', 'rating']).append(sugg_data, ignore_index=True)
        i += 1
    sugg_data['rating'] = sugg_data['rating'].apply(lambda x: int(x))
    sugg_data = sugg_data.sort_values(['rating'], ascending=[0])
    sugg_data = sugg_data[:number]
    #string = sugg_data['name']
    return list(sugg_data['name'])

In [26]:
data = pd.read_json('cook_dataset.json')
data['name'] = data['name'].apply(lambda x: x.lower().replace('ё', 'е'))
data.head()

,cuisine,ingredients,name,rating
0,francuzskaya,"[Яблоко, Финики, Мед, Лимонный сок, Ваниль, Со...",коблер,343
1,kitayskaya,"[Филе куриной грудки, Апельсины, Соевый соус, ...",куриные кусочки в апельсиновом соусе с чили,13
2,kitayskaya,"[Белое сухое вино, Соевый соус, Чеснок, Жидкий...",курица в кисло-сладком маринаде,66
3,armyanskaya,"[Вода, Пшеничная мука, Соль, Сливочное масло, ...",армянский лаваш,569
4,kitayskaya,"[Стебель сельдерея, Зеленый лук перья, Соевый ...",салат из сельдерея по‑китайски,62


In [33]:

my_str = ', '.join(list(data[data.name=='коблер'].ingredients)[0])
my_str

'Яблоко, Финики, Мед, Лимонный сок, Ваниль, Соль, Молотый имбирь, Кешью, Пекан, Кокосовое масло'

In [7]:
root = TrieNode('*')

In [8]:
for i in data['name']:
    add(root, i)

In [19]:
suggest = suggester(root,"салат")
type(suggest)

list

In [21]:
rating(root, 'салат', 10, data)

['салат из красной фасоли с творожным сыром…',
 'салат «цезарь» с чесночным майонезом',
 'салат с перепелиными яйцами, семгой и томатами…',
 'салат из зеленой фасоли с яйцами',
 'салат с моцареллой и руколой',
 'салат с семгой и перепелиными яйцами',
 'салат с авокадо и креветками',
 'салат «цезарь» с куриной грудкой',
 'салат из крабовых палочек с кукурузой',
 'салат из авокадо, томатов и базилика']

In [9]:
rating(root, 'ХАЧАП', 10)

,name,rating
4,хачапури с зеленью,2416
10,хачапури по‑аджарски.,2276
13,хачапури по‑грузински,1843
2,хачапури творожные,1274
7,хачапури по‑быстрому,879
5,хачапури с сулугуни,855
21,хачапури на кефире,810
20,хачапури по‑аджарски с сыром сулугуни и яйцом,688
17,хачапури по‑тбилисски за 5 минут,183
16,хачапури по‑летнему,151
